# Ollama + OpenAI + Python

## 1. Modellnamen angeben

Falls Sie ein anderes Modell als "phi3:mini" verwendet haben, ändern Sie den Wert in der Zelle unten. Diese Variable wird im gesamten Notebook im Code verwendet.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Einrichtung des OpenAI-Clients

Normalerweise wird der OpenAI-Client mit OpenAI.com oder Azure OpenAI verwendet, um mit großen Sprachmodellen zu interagieren.  
Er kann jedoch auch mit Ollama genutzt werden, da Ollama einen OpenAI-kompatiblen Endpunkt unter "http://localhost:11434/v1" bereitstellt.


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Erstelle eine Chat-Antwort

Nun können wir das OpenAI SDK verwenden, um eine Antwort für ein Gespräch zu generieren. Diese Anfrage sollte ein Haiku über Katzen erstellen:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Prompt-Engineering

Die erste Nachricht, die an das Sprachmodell gesendet wird, wird als "Systemnachricht" oder "System-Prompt" bezeichnet und legt die allgemeinen Anweisungen für das Modell fest.  
Du kannst deinen eigenen benutzerdefinierten System-Prompt bereitstellen, um ein Sprachmodell dazu zu bringen, Ausgaben auf eine andere Weise zu generieren.  
Ändere die `SYSTEM_MESSAGE` unten, um wie dein Lieblingscharakter aus einem berühmten Film oder einer TV-Serie zu antworten, oder lass dich von anderen System-Prompts auf [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts) inspirieren.

Sobald du die Systemnachricht angepasst hast, stelle die erste Benutzerfrage in der `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Beispiele mit wenigen Vorgaben

Eine weitere Möglichkeit, ein Sprachmodell zu steuern, besteht darin, ihm "wenige Vorgaben" zu geben – eine Reihe von Beispiel-Fragen und -Antworten, die zeigen, wie es reagieren soll.

Das folgende Beispiel versucht, ein Sprachmodell dazu zu bringen, wie ein Lehrassistent zu agieren, indem es einige Beispiele für Fragen und Antworten gibt, die ein Lehrassistent möglicherweise geben würde, und anschließend das Modell mit einer Frage auffordert, die ein Student stellen könnte.

Probieren Sie es zuerst aus und ändern Sie dann die `SYSTEM_MESSAGE`, `EXAMPLES` und `USER_MESSAGE` für ein neues Szenario.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Abruf-unterstützte Generierung

RAG (Retrieval Augmented Generation) ist eine Technik, um ein Sprachmodell dazu zu bringen, Fragen für einen bestimmten Bereich genau zu beantworten. Dabei wird zunächst relevante Information aus einer Wissensquelle abgerufen und anschließend eine Antwort basierend auf diesen Informationen generiert.

Wir haben eine lokale CSV-Datei mit Daten über Hybridautos bereitgestellt. Der untenstehende Code liest die CSV-Datei, sucht nach Übereinstimmungen mit der Benutzerfrage und generiert dann eine Antwort basierend auf den gefundenen Informationen. Beachten Sie, dass dies länger dauert als die vorherigen Beispiele, da mehr Daten an das Modell gesendet werden. Falls Sie feststellen, dass die Antwort immer noch nicht auf den Daten basiert, können Sie System-Engineering ausprobieren oder andere Modelle verwenden. Im Allgemeinen ist RAG effektiver mit entweder größeren Modellen oder mit feinabgestimmten Versionen von SLMs.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)


---

**Haftungsausschluss**:  
Dieses Dokument wurde mit dem KI-Übersetzungsdienst [Co-op Translator](https://github.com/Azure/co-op-translator) übersetzt. Obwohl wir uns um Genauigkeit bemühen, beachten Sie bitte, dass automatisierte Übersetzungen Fehler oder Ungenauigkeiten enthalten können. Das Originaldokument in seiner ursprünglichen Sprache sollte als maßgebliche Quelle betrachtet werden. Für kritische Informationen wird eine professionelle menschliche Übersetzung empfohlen. Wir übernehmen keine Haftung für Missverständnisse oder Fehlinterpretationen, die sich aus der Nutzung dieser Übersetzung ergeben.
